In [ ]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import opendatasets as od
import os
import glob
import xml.etree.ElementTree as ET
import json
import random
from pathlib import Path
import shutil
from google.colab import files
import yaml

In [ ]:
#download data from kaggle, please enter your username and key
od.download("https://www.kaggle.com/datasets/mahmoudelnazer/firedetection2")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mahmoudelnazer
Your Kaggle Key: ··········


100%|██████████| 29.7M/29.7M [00:03<00:00, 8.13MB/s]


Converting xml files to txt for Yolov5

In [ ]:
os.rename('/content/firedetection2/Firedetection/images/images','/content/firedetection2/Firedetection/images/Fire')

In [ ]:
classes= os.listdir('/content/firedetection2/Firedetection/images/')
classes

['Fire']

In [ ]:
def xml_to_yolo_bbox(bbox, w, h):
    x_center = ((bbox[0] + bbox[1]) / 2) / w
    y_center = ((bbox[2] + bbox[3]) / 2) / h
    width = ((bbox[1] - bbox[0]) / w)
    height = ((bbox[3] - bbox[2]) / h)
    return [x_center, y_center, width, height]

In [ ]:
label_files=('/content/firedetection2/Firedetection/labels/labels')
main_image_folder="/content/firedetection2/Firedetection/images"
main_label_folder="/content/firedetection2/Firedetection/labels"
images_folders=glob.glob("/content/firedetection2/Firedetection/images/*")
new_labels=('/content/firedetection2/Firedetection/labels/new_labels')

if not os.path.exists(new_labels):
        os.makedirs(new_labels)

In [ ]:
no_label=0
for perclass in images_folders:
  imagesperclass=os.listdir(perclass)
  for fil in imagesperclass:
      file_name=fil[:-4]
      label_file=os.path.join(label_files,file_name+'.xml')


      result = []

      # parse the content of the xml file
      if not os.path.exists(label_file):
        no_label+=1

        continue
      tree = ET.parse(label_file)
      root = tree.getroot()
      width = int(root.find("size").find("width").text)
      height = int(root.find("size").find("height").text)

      for obj in root.findall('object'):
          label = obj.find("name").text
          # check for new classes and append to list
          class_name=os.path.split(perclass)[-1]
          index = classes.index(class_name)
          pil_bbox = [int(x.text) for x in obj.find("bndbox")]
          yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
          # convert data to string
          bbox_string = " ".join([str(x) for x in yolo_bbox])
          result.append(f"{index} {bbox_string}")

      if result:
          # generate a YOLO format text file for each xml file
          with open(os.path.join(new_labels, f"{file_name}.txt"), "w", encoding="utf-8") as f:
              f.write("\n".join(result))
print(f"Number of images that doesn't have labels = {no_label}")

Number of images that doesn't have labels = 0


In [ ]:
!rm -r /content/firedetection2/Firedetection/labels/labels

In [ ]:
for folder in images_folders:
  images=os.listdir(folder)
  for image in images:
    os.rename(os.path.join(folder,image), os.path.join(main_image_folder,image))
!rm -r '/content/firedetection2/Firedetection/images/Fire'

In [ ]:
all_images=glob.glob("/content/firedetection2/Firedetection/images/*.jpg")
random.shuffle(all_images)

In [ ]:
all_images

['/content/firedetection2/Firedetection/images/ck0u13mbhte0v0838nxz2zdkf_jpeg.rf.ea7ac3c4de6eb11b8eb611eb4607088c.jpg',
 '/content/firedetection2/Firedetection/images/ck0negpok9ao50721aw0ozvpl_jpeg.rf.a38e1ae1ffb81f88c8e7b13052a3e043.jpg',
 '/content/firedetection2/Firedetection/images/ck0u01dbyuoe30701p6pfzb0q_jpeg.rf.dc34aba657747af88bf1eec63f19ea6b.jpg',
 '/content/firedetection2/Firedetection/images/ck0t30z1wlpg60838dc6e4lcx_jpeg.rf.d04e3521ec190c980d01cdf80d00183d.jpg',
 '/content/firedetection2/Firedetection/images/ck0nevwgw73ne094401pqo8zp_jpeg.rf.6afb2f6c249a5ab268fd7a8b61b29ea0.jpg',
 '/content/firedetection2/Firedetection/images/ck0qb93azjo480721ojf86ytj_jpeg.rf.0658191ccf3ef3dc7533eca8568b60d1.jpg',
 '/content/firedetection2/Firedetection/images/ck0m0dejchvnj08381f1xtpqs_jpeg.rf.90bdfa908276f45cb20d30ebb225c40a.jpg',
 '/content/firedetection2/Firedetection/images/ck0ujirqowixt07012814w3ox_jpeg.rf.af7d2c5ce6bcfedb6ffd891231b412c8.jpg',
 '/content/firedetection2/Firedetection/

Creating train and valid folders for traning

In [ ]:

Path("/content/data/train/images").mkdir(parents=True, exist_ok=True)
Path("/content/data/train/labels").mkdir(parents=True, exist_ok=True)
Path("/content/data/valid/images").mkdir(parents=True, exist_ok=True)
Path("/content/data/valid/labels").mkdir(parents=True, exist_ok=True)

In [ ]:
#split data 75% train and 25% valid
split=int(len(all_images)*0.75)
train_images=all_images[:split]
valid_images=all_images[split:]

In [ ]:
for image in train_images:
  name_alone=os.path.split(image)[-1]
  os.rename(image,os.path.join("/content/data/train/images",name_alone))
for image in valid_images:
  name_alone=os.path.split(image)[-1]
  os.rename(image,os.path.join("/content/data/valid/images",name_alone))

for label in train_images:
  name_alone=os.path.split(label)[-1][:-4]+'.txt'
  if not os.path.exists(os.path.join(new_labels,name_alone)):
    continue
  os.rename(os.path.join(new_labels,name_alone),os.path.join("/content/data/train/labels",name_alone))

for label in valid_images:
  name_alone=os.path.split(label)[-1][:-4]+'.txt'
  if not os.path.exists(os.path.join(new_labels,name_alone)):
    continue
  os.rename(os.path.join(new_labels,name_alone),os.path.join("/content/data/valid/labels",name_alone))

In [ ]:
#create yaml file
yaml_data={"train": "../data/train/images","val": "../data/valid/images", "nc":len(classes),"names":list(classes)}
with open('/content/data/data.yaml', 'w') as f:
        yaml.dump(yaml_data, f)

In [ ]:
!rm -r /content/firedetection2

In [ ]:
# to download the dataset
shutil.make_archive('data', 'zip', '/content/data')
#files.download("/content/data.zip")

'/content/data.zip'

Train the model

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 14130, done.
remote: Total 14130 (delta 0), reused 0 (delta 0), pack-reused 14130
Receiving objects: 100% (14130/14130), 13.47 MiB | 11.34 MiB/s, done.
Resolving deltas: 100% (9717/9717), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 31.9 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 1.6 MB 49.2 MB/s 


In [ ]:
!cp /content/data/data.yaml /content/yolov5/data

In [ ]:
!python train.py --img 640 --batch 16 --epochs 100 --data data.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-4-g7398d2d Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, deg

In [ ]:
!python detect.py --source /content/doun2.png  --weights /content/yolov5/runs/train/exp/weights/best.pt


detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/doun2.png, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-4-g7398d2d Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/doun2.png: 448x640 1 Fire, 14.6ms
Speed: 0.5ms pre-process, 14.6ms inference, 1.6ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp2
